<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Modulo_4/Projeto_final_modulo_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports e Dependências

In [2]:
import pandas as pd
import requests

#Funções ETL

In [12]:
#Treat numeric values from dataframe
def str_to_num(df_aux, index):
  for i in index:
    col_name = df_aux.columns[i]
    df_aux[col_name] = df_aux[col_name].str.replace('.', '')
    df_aux[col_name] = df_aux[col_name].str.replace(',', '.')
    df_aux[col_name] = pd.to_numeric(df_aux[col_name])
  return df_aux


In [60]:
def get_data(url, id):
  url = (url)
  response = requests.get(url)
  print("Request: {} => {}".format(id, response))
  if response.status_code == 200:
    payload = response.json()
  else:
    print(url)
    print(f'Error {response.status_code}: {response.reason}')
  return payload

#Return dataframe movies/series from api informed
def retrieve_data_from_api(url, id):
  payload = get_data(url, id)
  return pd.DataFrame(payload["results"]), payload["page"], payload["total_pages"]

#Return dataframe cast (artists and directors) from api informed
def retrieve_data_cast_from_api(url, id):
  payload = get_data(url, id)
  return pd.DataFrame(payload["cast"]), pd.DataFrame(payload["crew"])

#Return dataframe details:
# TV:{ genres, last_air_date, number_of_episodes, number_of_seasons, status }
# Movie: { budget, genres, runtime, status }
def retrieve_data_detail_from_api(url, id, api_variables):
  payload = get_data(url, id)
  df_aux = pd.DataFrame(columns=api_variables)
  for x in api_variables:
    if x == 'genres':
      df_genres = pd.DataFrame(payload["genres"])
    else:
      df_aux.loc[0, x] = payload[x]
  return df_aux, df_genres

In [14]:
#Reference: https://www.themoviedb.org/documentation/api
BASE_URL = "https://api.themoviedb.org/3/"
API_KEY = "?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6"
DEFAULT_LANGUAGE = "&language=en-US"
PAGE = "&page="

def build_themoviedb_url(request_type, page):
  uri = BASE_URL + request_type + API_KEY + DEFAULT_LANGUAGE 
  if page > 0:
    uri = uri + PAGE + str(page)
  return uri

#http://api.themoviedb.org/3/movie/550/casts?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6
#https://api.themoviedb.org/3/tv/88040/credits?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6
def build_themoviedb_cast_url(request_type, api_variable, movie_id):
  uri = BASE_URL + request_type +  str(movie_id) + api_variable + API_KEY
  return uri

#https://api.themoviedb.org/3/movie/500?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6&language=en-US
def build_themoviedb_details_url(request_type, movie_id):
  uri = BASE_URL + request_type +  str(movie_id) + API_KEY
  return uri

In [136]:
def load_all_themoviedb(request):
  print(request)
  current_page = 0
  total_pages = 1
  while current_page <= total_pages:
    uri = build_themoviedb_url(request, current_page)
    df_aux, current_page, total_pages = retrieve_data_from_api(uri, current_page)
    if current_page == 1:
      print("TOTAL: ", total_pages)
      df = pd.DataFrame(df_aux)
    else:
      df = df.append(df_aux, ignore_index=True)
    current_page = current_page+1
  print("DONE!")
  return df

def load_all_cast_themoviedb(request, api_variable, df_aux):
  for index, row in df_aux.iterrows():
    movie_id = row['id']
    uri = build_themoviedb_cast_url(request, api_variable, movie_id)
    df_art, df_dir = retrieve_data_cast_from_api(uri, movie_id)
    #get Acting department
    artists = get_department(df_art, 'known_for_department', 'Acting')
    #get Directing department
    directors = get_department(df_dir, 'department', 'Directing')
    df_aux.loc[index, 'Casting'] = get_string_of_series(artists)
    df_aux.loc[index, 'Crew'] = get_string_of_series(directors)
  return df_aux

def load_details_themoviedb(request, api_variables, df_aux):
  df_details = pd.DataFrame(columns=api_variables)
  for index, row in df_aux.iterrows():
    movie_id = row['id']
    uri = build_themoviedb_details_url(request, movie_id)
    df_det, df_genres = retrieve_data_detail_from_api(uri, movie_id, api_variables)
    #get name of genres
    if 'name' in df_genres:
      genres = get_string_of_series(df_genres['name'])
    else:
      genres = ''
    df_det.loc[0, 'genres'] = genres
    #append details of movie or serie
    df_details = df_details.append(df_det, ignore_index=True)
  #merge with main dataframe  
  df_aux = df_aux.merge(df_details, on='id')
  return df_aux

def get_department(df, column, department):
  if column in df:
    aux = df[df[column] == department]
    return aux['name']
  else:
    return ''

def get_string_of_series(serie):
  aux = ''
  for x in serie:
    aux = aux + x + ';'
  return aux[:-1]


In [16]:
def get_netflix_dataset():
  !wget https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/netflix_titles.csv?raw=true
  !ls -la
  #rename
  !mv 'netflix_titles.csv?raw=true' 'netflix_titles.csv'
  df = pd.read_csv('./netflix_titles.csv', sep = ",", usecols = ["type", "title", "director", "cast", "country", "date_added", "release_year", "rating", "duration", "listed_in", "description"])
  return df

##Métodos para checagem de **outliers**

In [17]:
def tukeys_method(df):
  # Parâmetros: series
  q1 = np.quantile(df, 0.25)
  q3 = np.quantile(df, 0.75)
  iqr = q3-q1
  inner_fence = 1.5*iqr
  outer_fence = 3*iqr

  #inner fence lower and uper end
  inner_fence_le = q1-inner_fence
  inner_fence_ue = q3+inner_fence

  #outer fence lower and uper end
  outer_fence_le = q1-outer_fence
  outer_fence_ue = q3+outer_fence

  outliers_prob = []
  outliers_poss = []
  for index, x in enumerate(df):
    if x <= outer_fence_le or x >= outer_fence_ue:
      outliers_prob.append(index)
  for index, x in enumerate(df):
    if x <= inner_fence_le or x >= inner_fence_ue:
      outliers_poss.append(index)
  return outliers_prob, outliers_poss

#Extração dos dados

1.   Filmes - [the movie database](https://www.themoviedb.org/)
2.   Série - [the movie database](https://www.themoviedb.org/)
3.   Netflix - [Kaggle](https://www.kaggle.com/shivamb/netflix-shows)

In [ ]:
#Movies
df_movies = load_all_themoviedb("movie/top_rated")
df_movies.info()

In [ ]:
#Series
df_series = load_all_themoviedb("tv/top_rated")
df_series.info()

In [ ]:
#Netflix
df_netflix = get_netflix_dataset()
df_netflix.info()

##Extração de Dados de filmes e séries



###**Elenco:**
*   Atores
*   Diretores



In [127]:
#Movies
df_movies_cast = load_all_cast_themoviedb("movie/", "/casts", df_movies)
df_movies_cast

Request: 761053 => <Response [200]>
Request: 724089 => <Response [200]>
Request: 19404 => <Response [200]>
Request: 441130 => <Response [200]>
Request: 278 => <Response [200]>
Request: 696374 => <Response [200]>
Request: 238 => <Response [200]>
Request: 424 => <Response [200]>
Request: 372058 => <Response [200]>
Request: 240 => <Response [200]>
Request: 644479 => <Response [200]>
Request: 129 => <Response [200]>
Request: 572154 => <Response [200]>
Request: 496243 => <Response [200]>
Request: 497 => <Response [200]>
Request: 556574 => <Response [200]>
Request: 755812 => <Response [200]>
Request: 680 => <Response [200]>
Request: 637 => <Response [200]>
Request: 389 => <Response [200]>
Request: 13 => <Response [200]>
Request: 155 => <Response [200]>
Request: 122 => <Response [200]>
Request: 11216 => <Response [200]>
Request: 769 => <Response [200]>
Request: 429 => <Response [200]>
Request: 311 => <Response [200]>
Request: 532067 => <Response [200]>
Request: 630566 => <Response [200]>
Requ

KeyboardInterrupt: ignored

In [ ]:
#Series
df_series_cast = load_all_cast_themoviedb("tv/", "/credits", df_series)
df_series_cast

###**Classificação e demais informações**
Filmes:
*   Gênero
*   Budget
*   Duração
*   Status

Séries:
*   Gênero
*   Última data no ar
*   Número de episódios
*   Número de temporadas
*   Status

In [ ]:
df_movies_details = load_details_themoviedb("movie/", ["id","budget", "genres", "runtime", "status"], df_movies)
df_movies_details

In [140]:
df_series_details = load_details_themoviedb("tv/", ["id", "genres", "last_air_date", "number_of_episodes", "number_of_seasons", "status"], df_series)
df_series_details

Request: 100 => <Response [200]>
Request: 88040 => <Response [200]>
Request: 83097 => <Response [200]>
Request: 100049 => <Response [200]>
Request: 83095 => <Response [200]>
Request: 80564 => <Response [200]>
Request: 99071 => <Response [200]>
Request: 61663 => <Response [200]>
Request: 93019 => <Response [200]>
Request: 96316 => <Response [200]>
Request: 68129 => <Response [200]>
Request: 73055 => <Response [200]>
Request: 96150 => <Response [200]>
Request: 65648 => <Response [200]>
Request: 65930 => <Response [200]>
Request: 67389 => <Response [200]>
Request: 87432 => <Response [200]>
Request: 83121 => <Response [200]>
Request: 72636 => <Response [200]>
Request: 68188 => <Response [200]>
Request: 90447 => <Response [200]>
Request: 68349 => <Response [200]>
Request: 60863 => <Response [200]>
Request: 85937 => <Response [200]>
Request: 95269 => <Response [200]>
Request: 77721 => <Response [200]>
Request: 45950 => <Response [200]>
Request: 84669 => <Response [200]>
Request: 76121 => <Re

,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count,genres,last_air_date,number_of_episodes,number_of_seasons,status
0,None,2004-05-10,"[16, 35]",100,I Am Not an Animal,[GB],en,I Am Not an Animal,I Am Not An Animal is an animated comedy serie...,12.076,/qG59J1Q7rpBc1dvku4azbzcqo8h.jpg,9.4,598,Animation;Comedy,2004-06-14,6,1,Ended
1,/7gbmM2NWcqZONbp65HUWDf4wr0Q.jpg,2019-07-12,"[16, 18]",88040,Given,[JP],ja,ギヴン,"Tightly clutching his Gibson guitar, Mafuyu Sa...",28.498,/pdDCcAq8RNSZNk81PXYoHNUPHjn.jpg,9.2,391,Animation;Drama,2019-09-20,11,1,Ended
2,/uAjMQlbPkVHmUahhCouANlHSDW2.jpg,2019-01-11,"[16, 9648, 10765, 10759, 18]",83097,The Promised Neverland,[JP],ja,約束のネバーランド,"Surrounded by a forest and a gated entrance, t...",65.812,/yxdeII5tI8qqiERcMxjW9DfB6Gz.jpg,9.2,545,Animation;Mystery;Sci-Fi & Fantasy;Action & Ad...,2021-02-26,23,2,Returning Series
3,/rnf2BDKeF1sxPEPhAcdvL0auuxr.jpg,2020-10-03,"[16, 35]",100049,TONIKAWA: Over the Moon for You,[JP],ja,トニカクカワイイ,"First comes marriage, then comes an earthbound...",32.649,/n1oOuXlcHF1brgcgyiwmnIODXso.jpg,9.1,176,Animation;Comedy,2020-12-19,12,1,Ended
4,/qSgBzXdu6QwVVeqOYOlHolkLRxZ.jpg,2019-01-09,"[16, 10759, 10765, 18]",83095,The Rising of the Shield Hero,[JP],ja,盾の勇者の成り上がり,Iwatani Naofumi was summoned into a parallel w...,15.476,/6cXf5EDwVhsRv8GlBzUTVnWuk8Z.jpg,9.1,457,Animation;Action & Adventure;Sci-Fi & Fantasy;...,2019-06-26,25,2,Returning Series
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,/kY8ciI1pR2sJrkZeGpR1gNxxGfB.jpg,2000-07-05,[10764],10160,Big Brother,[US],en,Big Brother,American version of the reality game show whic...,24.067,/3z1eKtApXsr0FbQAoHX0F8ATwZO.jpg,4.9,173,Reality,2020-10-28,783,22,Returning Series
1657,/qnTBxxv7q4xVwgZBLb0vtILLelh.jpg,2007-10-14,[10764],14814,Keeping Up with the Kardashians,[US],en,Keeping Up with the Kardashians,A peek inside the exploits and privileged priv...,51.804,/l7QHRrX1EYgQAzNJdCdoEQHoHJ.jpg,4.8,226,Reality,2020-11-12,270,20,Returning Series
1658,/dHlKH9vTumj7tPBeYVWGzI7UJ9I.jpg,1975-04-05,"[10759, 10765, 9648, 18]",45253,Super Sentai,[JP],ja,スーパー戦隊シリーズ,The Super Sentai Series is the name given to t...,29.646,/tDq8av51oHR8YWgkiHQ2oeRO5iL.jpg,4.7,104,Action & Adventure;Sci-Fi & Fantasy;Mystery;Drama,2021-02-21,2212,44,Returning Series
1659,/hUz0UHWSVd6e1g3I6vT4shAsnoY.jpg,1985-02-19,"[10766, 18]",1871,EastEnders,[GB],en,EastEnders,The everyday lives of working-class residents ...,27.180,/z4jgyI5TpoRZiJTNchkVkMrGQyz.jpg,3.8,165,Soap;Drama,2021-02-26,6155,37,Returning Series
